In [15]:
import cv2 as cv2
from matplotlib import pyplot as plt
import scipy.signal as sg 
import scipy.ndimage as sc
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['image.cmap'] = 'gray'
from src.image_utils import open_image
# why the cmap argument? : 
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

ModuleNotFoundError: No module named 'src'